In [3]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## Selection


Today's agenda:

- Randomized Linear Time Selection

Given an unsorted list $a$ and an integer $k$ ($0\leq k< |a|$), the **order statistics** problem asks us to return the $k$th smallest element from $a$. We also refer to the $k$th smallest element the element of *rank* $k$.  

Example: Let $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle$. For $k=0$, the "$0$th smallest" element is the minimum element in $a$, or $-1$. For $k=n-1$, it is the maximum, or $99$. For $k=3$, we return $3$.

Before we come up with a randomized algorithm, we can make a couple of simple observations.

First, any algorithm for this problem requires $\Omega{n}$ work. Why?

Second, we can reduce this problem to sorting as follows. We sort $a$ and return the $k$th element of this sorted list. This requires $O(n\log n)$ time. 

Can we do any better? Sorting seems like overkill since we don't really need to rearrange all the elements, or even return a list.

A useful observation is that the $k$th smallest element in $a$ *partitions* $a$ into a set of $k-1$ smaller elements, and a set of $n-k-1$ larger elements. 

Example: Suppose $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle$ and $k=3$. So $3$ is larger than $\langle 2, 1, -1 \rangle$ and smaller than $\langle5, 4, 99\rangle$.

Notice that *for any* element $x$ in the list, we can look at each element in the list to compute the rank of $x$. This can be done in $O(n)$ work and $O(\log n)$ span. 

Example: Suppose $a=\langle 2, 5, 4, 1, 3, -1, 99\rangle$ and $k=3$. So $a[0] = 2$ is larger than 2 elements ($\langle 1, -1 \rangle$) and smaller than 5 elements ($\langle 5, 4, 3, 99\rangle$).

We can see from this example that once we've identified 2 smaller elements, the element of rank $k=3$ in $a$ must be in $\langle5, 4, 3, 99\rangle$. Moreover it's rank is $k-2 = 1$ in this list. 

This is a little like binary search, but with the partition step helping establish some order.


This observation the following recursive algorithm.

<p>\[\begin{array}{ll}  
\mathit{simple-select}~a~k =   
\\  
\texttt{let}  
\\  
~~~~p = a[0]   
\\  
~~~~\ell = \left\langle\, x \in a \;|\; x < p \,\right\rangle  
\\  
~~~~r = \left\langle\, x \in a \;|\; x > p \,\right\rangle  
\\  
\texttt{in}  
\\  
~~~~\texttt{if}~(k < |\ell|)~\texttt{then}~\mathit{select}~\ell~k  
\\  
~~~~\texttt{else if}~(k < |a| - |r|)~\texttt{then}~p  
\\  
~~~~\texttt{else}~\mathit{select}~r~(k - (|a| - |r|))  
\\  
\texttt{end}  
\end{array}\]</p>

We just have one recursive call so no parallelism there. However, we can use filter to partition in parallel. This has $O(|a|)$ work $O(\log |a|)$ span.

What is the total work over all recursions? We know that the work in each recursive call is the $\max\{W(\mid l\mid, W(\mid r\mid\} + O(n)$. 

Consider the case where $a$ is a sorted list. Then in every call, $l = \emptyset$, and $\mid r\mid = n-1$. Thus we have $W(n) = W(n-1) + n = O(n^2)$. This is worse than just sorting the list!


The problem is that we just don't know anything about the element we're using for the partition. How do we avoid this worst case?

Pick a random element, or **pivot**, for partitioning!

<p>\[\begin{array}{ll}  
\mathit{select}~a~k =   
\\  
\texttt{let}  
\\  
~~~~p = \mbox{pick a uniformly random element from}~a   
\\  
~~~~\ell = \left\langle\, x \in a \;|\; x < p \,\right\rangle  
\\  
~~~~r = \left\langle\, x \in a \;|\; x > p \,\right\rangle  
\\  
\texttt{in}  
\\  
~~~~\texttt{if}~(k < |\ell|)~\texttt{then}~\mathit{select}~\ell~k  
\\  
~~~~\texttt{else if}~(k < |a| - |r|)~\texttt{then}~p  
\\  
~~~~\texttt{else}~\mathit{select}~r~(k - (|a| - |r|))  
\\  
\texttt{end}  
\end{array}\]</p>

Notice that the probability of the worst-case is vanishingly small. 

The size of the $l$ and $r$ will depend on the random choice. Thus the recurrences describing the work and span depend on each random choice and we need to find their expected asymptotic.

Let's get some intution for what's happening. We saw that the work of our algorithm depends on $\max\{W(\mid l\mid, W(\mid r\mid\}$ in each recursive call. While there is only a $1/n$ probability of choosing a balanced split, we can observe that any constant fraction reduction in the size of the larger list yields good performance. 

So, suppose we knew that $\max\{W(\mid l\mid, W(\mid r\mid\} \leq W(3n/4)$. This would be a good enough split since the overall work would be $W(n) = W(3n/4) + n = O(n)$.

We can examine where $p$ might land in the sorted version of $a$, to understand the probability of a good split.

![selection-intuition.jpg](selection-intuition.jpg)


We can see that $\mathbf{P}[\max\{W(\mid l\mid, W(\mid r\mid\} \leq W(3n/4)]\geq1/2$.

If we think of each choice of pivot as a coin flip ("good" vs. "bad") then the expected number of pivot choices to reduce the input to $3n/4$ is 2. In other words, every two recursions yields the desired reduction in list size, and so in expectation we will do linear work.